In [ ]:
# %load_ext lab_black
import json
import os
import sys
from time import sleep

from  pythagoras import PolyphonicPlayer

import numpy as np
from scipy import signal
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from ipywidgets import interact, IntSlider

plt.style.use("dark_background")
# np.set_printoptions(precision=2, linewidth=150)
base_layout = dict(
    template="plotly_dark",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    margin=dict(l=20, r=20, t=20, b=20),
)

In [ ]:
nperseg = 1024 * 32
# filename = "data/moon_river_chord1.wav"
# filename = "data/fa1.wav"
filename = "data/ther1.wav"
# filename = "data/moon_river_clipped.wav"
# filename = "data/father_clipped.wav"
rate, data = wav.read(filename)
data = data[:, 0] #+ data[:, 1]  # convert to mono


In [ ]:
# f, t, Zxx = signal.stft(data, rate, nperseg=nperseg)

# clip = nperseg // 64
# Zxx = Zxx[:clip]
# f = f[:clip]
# print(Zxx.shape)

# # TODO use only plotly
# _, ax = plt.subplots(figsize=(20, 6))

# ax.pcolormesh(t, f, np.abs(Zxx), vmin=0, shading="gouraud")
# ax.set_title("STFT Magnitude")
# ax.set_ylabel("Frequency [Hz]")
# ax.set_xlabel("Time [sec]")

In [ ]:
# freq = 1000
# t = np.linspace(0, len(data) / rate, len(data))
# fake_data = np.sin(2 * np.pi * freq * t)

In [ ]:
lowest_frequency = 40
omega = 1000
octaves = 6

# resolution of the human ear is about 6 cents = 1/16 of a semitone
frequencies = lowest_frequency * 2 ** np.arange(0, octaves, 1/12/16)   # this should catch all the differences detectable by the human ear
widths = omega * rate / (2 * np.pi * frequencies)
wavelet_spectrum = signal.cwt(data, signal.morlet2, widths, w=omega)

In [ ]:
# # divide into chunks
# n = rate // 10
# num_of_chunks = wavelet_spectrum.shape[1] // n

# # wavelet_spectrum_abs = wavelet_spectrum_abs[:, :num_of_chunks * n]
# # chunk_means = wavelet_spectrum_abs.reshape(wavelet_spectrum_abs.shape[0], num_of_chunks, n).mean(axis=2)

# # times = np.arange(len(data)) / rate
# # times = times[::n][:num_of_chunks]
# # _, ax = plt.subplots(figsize=(20, 6))

# # ax.pcolormesh(times, frequencies, np.abs(chunk_means), vmin=0, shading="gouraud")
# # ax.set_title("Wavelet magnitude")
# # ax.set_ylabel("Frequencies [Hz]")
# # ax.set_xlabel("Time [sec]")

# ! plot total means
wavelet_spectrum_abs = np.abs(wavelet_spectrum)
total_means = wavelet_spectrum_abs.mean(axis=1)
# _, ax = plt.subplots(figsize=(20, 6))
# ax.plot(frequencies, total_means)
# # ax.plot(np.arange(len(frequencies)), total_means)   # this plots logarithmically
# ax.set_xlabel("Frequencies [Hz]")

In [ ]:
def get_peaks_cwt(fft, width_range=(6, 50), height=400):
    fft = np.abs(fft)
    widths = np.arange(width_range[0], width_range[1])
    peaks = signal.find_peaks_cwt(fft, widths)
    volumes = fft[peaks]
    mask = volumes > height
    return peaks[mask], volumes[mask]


def get_peaks(fft, window=17, height=400, distance=10, smooth=True):
    fft = np.abs(fft)
    original_fft = fft
    if smooth:
        fft = signal.savgol_filter(fft, window, 3)
    peaks, peaks_prop = signal.find_peaks(fft, height=height, distance=distance)
    signal.find_peaks_cwt
    # TODO maybe use prominences
    # heights = peaks_prop["peak_heights"]

    # get heights from the original fft, not the smooth one, but take the max from peak's surrounding, due to noise
    # heights = original_fft[peaks]
    peak_error_margin = 2
    heights = [max(original_fft[max(peak-peak_error_margin, 0) : peak+peak_error_margin]) for peak in peaks]
    return peaks, heights, fft


def save_peaks_and_volumes(peaks, volumes, name):
    npy_filename = os.path.join("chord_data", f"{name}.npy")

    with open(npy_filename, "wb") as f:
        np.save(f, (peaks, volumes))
        
        
def play(peaks, volumes, length):
    volumes = volumes / np.sum(volumes)

    freqs = frequencies[peaks]
    # print(len(peaks))
    # print(freqs)
    # print(volumes)

    player = PolyphonicPlayer(base_freq=1, max_voices=len(freqs))
    player.ratios = freqs
    player.volumes = volumes
    player.start()

    sleep(length)
    player.kill()
    player.join()

In [ ]:
fig = go.FigureWidget(layout=base_layout)
fig.update_layout(
    xaxis_range=[0, len(total_means)],
    yaxis_range=[0, max(total_means)],
    width=2000,
    height=300,
)
fig.add_scattergl()
fig.add_scattergl()


# player = PolyphonicPlayer(base_freq=1, max_voices=200)
# player.start()


@interact(height=(0, max(total_means) / 10), distance=(1, 40), window=(4,21), play=False)
def update(height, distance=10, window=5, play=False):
    global peaks, volumes
    # peaks, volumes = get_peaks_cwt(freq_slice)
    peaks, volumes, smooth = get_peaks(total_means, height=height, distance=distance, window=window)
    
    # player.ratios = frequencies[peaks]
    # if play:
    #     player.volumes = volumes / np.sum(volumes)
    # else:
    #     player.volumes = np.zeros(len(player.volumes))

    shapes = list()
    for peak, vol in zip(peaks, volumes):
        shapes.append(
            {
                "type": "line",
                "line_color": "orange",
                "x0": peak,
                "y0": 0,
                "x1": peak,
                "y1": vol,
            }
        )

    with fig.batch_update():
        fig.data[0].y = np.abs(total_means)
        fig.data[1].y = np.abs(smooth)
        fig.layout.shapes = shapes
    print(len(peaks))


fig

In [ ]:
player.kill()
player.join()

In [ ]:
play(peaks, volumes, 6)

In [ ]:
# name = os.path.split(filename)[1].split(".")[0]
name = "ther1"
save_peaks_and_volumes(frequencies[peaks], volumes, name)

In [ ]:
# peaks, volumes, smooth = get_peaks(total_means, height=30000, distance=14, window=5)
# # peaks, volumes = get_peaks_cwt(total_means, height=10000)
# peaks = frequencies[peaks]

# # plot total means
# _, ax = plt.subplots(figsize=(20, 6))
# ax.plot(frequencies, total_means)
# ax.plot(frequencies, smooth)
# ax.set_xlabel("Frequencies [Hz]")

# # add vertical lines to the plot for the peaks, with given heights
# for peak, volume in zip(peaks, volumes):
#     if peak > frequencies[-1]:
#         break
#     ax.axvline(peak, color="r", linewidth=2, alpha=0.5, ymax=volume / max(volumes))
#     # ax.axvline(peak, color="orange", linewidth=2, alpha=0.5)



In [ ]:
# !python play_peaks.py chord_data/moon_river_chord1.npy

In [ ]:
# fig = go.FigureWidget(layout=base_layout)
# fig.update_layout(
#     xaxis_range=[0, Zxx.shape[0]],
#     yaxis_range=[0, 4000],
#     width=1000,
#     height=300,
# )
# fig.add_scattergl()
# fig.add_scattergl(line_color="green")

# xrange = Zxx.shape[1] - 1


# @interact(x=(0, xrange))
# def update(x=xrange // 2):
#     freq_slice = Zxx[:, x]
#     peaks, heights, smooth = get_peaks(freq_slice)

#     shapes = list()
#     for peak, height in zip(peaks, heights):
#         shapes.append(
#             {
#                 "type": "line",
#                 "line_color": "orange",
#                 "x0": peak,
#                 "y0": 0,
#                 "x1": peak,
#                 "y1": height,
#             }
#         )

#     with fig.batch_update():
#         fig.data[0].y = np.abs(freq_slice)
#         fig.data[1].y = smooth
#         fig.layout.shapes = shapes


# fig

In [ ]:
def get_ratios(peaks):
    peaks = peaks.reshape((1, -1))
    return peaks / peaks.T


def get_strengths(heights):
    heights = heights.reshape((1, -1))
    return (heights * heights.T) ** (1 / 2)


# significant = get_strengths(heights) > 400
# significant = np.triu(significant)
# get_ratios(peaks) * significant

In [ ]:
get_ratios(peaks)

In [ ]:
str(heights)

In [ ]:
if not [1, 1]:
    print(2)

In [ ]:
[0] * 0

In [ ]:
f = go.FigureWidget(layout=base_layout)
f.update_layout(
    yaxis_range=[0, Zxx.shape[0]],
    xaxis_range=[0, Zxx.shape[0]],
    width=600,
    height=600,
)

f.add_scattergl(mode="markers")
f

In [ ]:
speedup = 1
time_len = 60  # in seconds, max 60
i_range = Zxx.shape[1] * time_len // 60

for i in range(i_range):
    freq_slice = Zxx[:, i]
    peaks, heights, _ = get_peaks(freq_slice)

    xs, ys = np.meshgrid(peaks, peaks)
    xs = xs.flatten()
    ys = ys.flatten()
    sizes = get_strengths(heights).flatten() / 50
    ratios = get_ratios(peaks)

    with f.batch_update():
        f.update_traces(x=xs, y=ys, marker_size=sizes)  # set marker_color
#     time.sleep(time_len / i_range / speedup)
print("done")